<a href="https://colab.research.google.com/github/MarcosRolando/OrgaDeDatosTP2/blob/main/neuralNetwoork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import regex as re
from tensorflow import feature_column


In [15]:
'''
df1 = pd.read_csv('/content/prediccionesFrioFrio (1).csv')
df2 = pd.read_csv('/content/prediccionesFrioFrio (2).csv')

dfMerge = df1
target = []

for i in df1['Opportunity_ID']:
  if df1.loc[df1['Opportunity_ID'] == i, 'Target'].item() >= 0.5:
    target.append(max(df1.loc[df1['Opportunity_ID'] == i, 'Target'].item(), df2.loc[df2['Opportunity_ID'] == i, 'Target'].item()))
  else:
    target.append(max(df1.loc[df1['Opportunity_ID'] == i, 'Target'].item(), df2.loc[df2['Opportunity_ID'] == i, 'Target'].item()))

dfMerge['Target'] = target

dfMerge.to_csv('prediccionesFrioFrioMerge.csv', index=False)
'''

"\ndf1 = pd.read_csv('/content/prediccionesFrioFrio (1).csv')\ndf2 = pd.read_csv('/content/prediccionesFrioFrio (2).csv')\n\ndfMerge = df1\ntarget = []\n\nfor i in df1['Opportunity_ID']:\n  if df1.loc[df1['Opportunity_ID'] == i, 'Target'].item() >= 0.5:\n    target.append(max(df1.loc[df1['Opportunity_ID'] == i, 'Target'].item(), df2.loc[df2['Opportunity_ID'] == i, 'Target'].item()))\n  else:\n    target.append(max(df1.loc[df1['Opportunity_ID'] == i, 'Target'].item(), df2.loc[df2['Opportunity_ID'] == i, 'Target'].item()))\n\ndfMerge['Target'] = target\n\ndfMerge.to_csv('prediccionesFrioFrioMerge.csv', index=False)\n"

In [16]:
df = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

#df = df[df['Brand'] == 'No']
'''
df['days'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']).dt.days
df['days'] = df.groupby('Opportunity_ID')['days'].transform('max')
df['days'] = df['days'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
df['days'] = df['days'] > 0

df['TRF'] = df.groupby('Opportunity_ID')['TRF'].transform('max')

df = df[(df['Has_Contract_Number'] == 'Yes')]
df['Ratio'] = df.groupby('Territory')['Stage'].transform('mean')
df[df['Ratio'] < 0.8]['Territory'].value_counts()'''
#df[df['Products'] == 'Product_Name_111']
df['Product_Family'].value_counts()


Product_Family_77     1345
Product_Family_133    1249
Product_Family_132     763
Product_Family_212     636
Product_Family_100     619
                      ... 
Product_Family_34        1
Product_Family_33        1
Product_Family_166       1
Product_Family_48        1
Product_Family_217       1
Name: Product_Family, Length: 227, dtype: int64

Tuneamos algunas cosas del DataFrame que notamos en el analisis de los datos.

In [17]:
def preprocess_dataframe(df):

  df.fillna(value=0, inplace=True) #Reemplazamos NAN por 0, ya que NAN rompe a Tensorflow

  #Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
  #Estos particularmente son whitespace, coma y parentesis por ejemplo.
  df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                    'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                    'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

  df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
  df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) #0 corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won. Asi tenemos un problema de clasificacion binario que puede entender la red neuronal.

  df.loc[:, 'Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], 'coerce',
                                                                  format='%m/%d/%Y')
  df.loc[:, 'Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df.loc[:, 'Last_Modified_Date'] = pd.to_datetime(df['Last_Modified_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df.loc[:, 'Quote_Expiry_Date'] = pd.to_datetime(df['Quote_Expiry_Date'], 'coerce', format='%m/%d/%Y')

  df = df[df['Opportunity_ID'] != 9773] #Hardcodeo este filtrado porque el id 9773 tiene mal cargada la fecha de delivery end, dando una diferencia de 200 anios xd"

  #Pongo .loc porque pandas me jode con warnings que son falsos positivos de slice copy"
  #Gracias Pandas!"

  #Creamos una nueva columna (Feature Engineering) que contiene la longitud en dias 
  #estimada de la operacion. En el informe habiamos encontrado que aparentaba haber
  #una relacion cuadratica de decrecimiento a medida que aumentaban los dias donde disminuia
  #la chance de completar la operacion.
  df['Planned_Opportunity_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']
  df.loc[:, 'Planned_Opportunity_Time'] = df['Planned_Opportunity_Time'].dt.days
  df['Planned_Opportunity_Time'] = df['Planned_Opportunity_Time'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
  df['Planned_Opportunity_Time'] = df.groupby('Opportunity_ID')['Planned_Opportunity_Time'].transform('max')

  df['End_Date_vs_Modified_Date_Difference'] = df['Planned_Delivery_End_Date'] - df['Last_Modified_Date']
  df.loc[:, 'End_Date_vs_Modified_Date_Difference'] = df['End_Date_vs_Modified_Date_Difference'].dt.days
  df['End_Date_vs_Modified_Date_Difference'] = df['End_Date_vs_Modified_Date_Difference'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
  df['End_Date_vs_Modified_Date_Difference'] = df.groupby('Opportunity_ID')['End_Date_vs_Modified_Date_Difference'].transform('max')

  df['Expiry_Date_vs_Modified_Date_Difference'] = df['Planned_Delivery_End_Date'] - df['Quote_Expiry_Date']
  df.loc[:, 'Expiry_Date_vs_Modified_Date_Difference'] = df['Expiry_Date_vs_Modified_Date_Difference'].dt.days
  df['Expiry_Date_vs_Modified_Date_Difference'] = df['Expiry_Date_vs_Modified_Date_Difference'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
  df['Expiry_Date_vs_Modified_Date_Difference'] = df.groupby('Opportunity_ID')['Expiry_Date_vs_Modified_Date_Difference'].transform('max')

  #Pasamos todo a dolares
  currency_conversion = {'AUD':0.707612, 'EUR':1.131064, 'GBP':1.318055, 'JPY':0.008987, 'USD':1.0}
  df['Total_Taxable_Amount_Currency'] = df[['Total_Taxable_Amount_Currency']].replace(currency_conversion)
  df['Total_Taxable_Amount'] = df['Total_Taxable_Amount_Currency'] * df['Total_Taxable_Amount']

  #Modifico la columna Brand para que en vez de decir que marca es, solo diga
  #si tiene o no marca. Es importante aclarar que verificamos que siempre que una oportunidad
  #tiene un producto con marca entonces todos sus productos tienen marca. Esto se cumple
  #tanto en el set de entrenamiento como en el de test, por lo tanto al hacer drop_duplicates
  #no nos va a pasar nunca el caso donde nos pudieramos quedar con una entrada de producto
  #sin marca mientras que algun otro producto si tuviera, ya que confirmamos que o todos tienen
  #marca o ninguno tiene.
  df.loc[df['Brand'] == 'None', 'Brand'] = 'No'
  df.loc[df['Brand'] != 'No', 'Brand'] = 'Yes'

  #Agrego una columna que indique la cantidad de productos que tiene esa
  #oportunidad
  df['Product'] = 1
  df['Product_Amount'] = df.groupby('Opportunity_ID')['Product'].transform(lambda x: x.sum())

  #Agrego una columna que indica si el owner de la cuenta es el mismo que el de la oportunidad
  #o no
  df['Same_Owner'] = (df['Account_Owner'] == df['Opportunity_Owner'])
  df['Same_Owner'] = df['Same_Owner'].replace({False:'No', True:'Yes'})

  #Agrego columna que indica si el ultimo que modifico la oportunidad es el mismo que el opportunity owner
  df['Same_Owner_Modifier'] = (df['Last_Modified_By'] == df['Opportunity_Owner'])
  df['Same_Owner_Modifier'] = df['Same_Owner_Modifier'].replace({False:'No', True:'Yes'})

  #Agrego una columna que indica si tiene o no fecha de expiracion
  df['Quote_Expiry_Date'] = (df['Quote_Expiry_Date'] != 'NaT')
  df.rename(columns={'Quote_Expiry_Date':'Has_Expiry_Date'}, inplace=True)
  df['Has_Expiry_Date'] = df['Has_Expiry_Date'].replace({True:'Yes',False:'No'})

  #Reemplazo las 4 columnas de aprobacion por solo 2 columnas que indiquen si tuvo la aprobacion
  #de delivery y burocratica o no. Recalco que si nunca la necesito seria equivalente a si
  #la necesito y la consiguio.
  df['Delivery_Approved'] = df['Pricing_Delivery_Terms_Quote_Appr'] + df['Pricing_Delivery_Terms_Approved']
  df['Delivery_Approved'] = df['Delivery_Approved'].replace({0:1, 1:0, 2:1})
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_0_Approval'] + df['Bureaucratic_Code_0_Approved']
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_Approved'].replace({0:1, 1:0, 2:1})
  df['Approved'] = df['Delivery_Approved'] & df['Bureaucratic_Code_Approved']

  #Cambio TRF por una columna que es el valor maximo de los TRF de la oportunidad
  df["TRF"] = df.groupby("Opportunity_ID")["TRF"].transform("max")

  def combineProducts(x):
    products = ""
    added = []
    for product in x:
      product = re.findall('\d+', product)[0]
      if added.count(product) == 0:
        products += product
        added.append(product)
    return products

  #Junto todos los productos en una sola entrada
  df['Products'] = df.groupby('Opportunity_ID')['Product_Family'].transform(combineProducts)

  #Pruebo volar duplicados, solo cambia el producto. Si el producto no importa
  #entonces volar duplicados no deberia importar. Obviamente vuelo el producto en el que
  #quede tambien.
  df.drop_duplicates('Opportunity_Name',inplace=True)
  df.drop(columns=['Product_Name','Product_Family','Opportunity_Name'],inplace=True)

  #Normalizo las columnas numericas
  normalized_columns = ['ASP_converted','TRF','Total_Taxable_Amount', 'Product_Amount',
                        'Planned_Opportunity_Time', 'End_Date_vs_Modified_Date_Difference',
                        'Expiry_Date_vs_Modified_Date_Difference']
  for column in normalized_columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

  #Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
  #Algunas columnas borradas son porque pienso que no tienen incidencia, ir viendo.
  #TODO: Analizar si el Sales_Contract_No no es que importe el numero en si, sino si tiene
  #o no tiene numero de contrato. Por ahora no lo meto como input.
  #TODO: Ver el mismo tema con la columna 'Price', la mayoria tiene None u Other
  #y solo unos pocos tienen precio numerico. Quiza importe que tenga precio o no tenga,
  #o si no tiene precio quiza importe si es None u Other. Por ahora no lo pongo
  #como input.
  df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                  'Prod_Category_A', 'ID', 'Opportunity_ID', 
                   'Actual_Delivery_Date'],inplace=True)

  #Drop columnas que quiza podamos usar pero por ahora no las uso
  df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                  'Last_Modified_Date',
                  'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                  'Month',
                  'Delivery_Year',
                  'Price','ASP','ASP_Currency','Total_Amount_Currency',
                  'Total_Amount','Total_Taxable_Amount_Currency', 'Currency',
                   'Product_Category_B','Last_Modified_By', 'Account_Owner',
                   'Opportunity_Owner','Account_Name','Product_Type','Size',
                   'Billing_Country',
                   'Approved', 'Sales_Contract_No',
                   'Product', 'Products','Quote_Type','Territory']
                   ,inplace=True)
  
  #Definimos que tipo de feature es cada columna

  #Debemos separar algunos de los registros para armar un set de test propio (no el de la catedra). De esta forma sabremos rapidamente
  #si nuestro modelo esta dando resultados optimos o no sin necesidad de estar subiendo el TP a Kaggle constantemente.
  #Sin embargo, no queremos usar tantos registros ya que estariamos disminuyendo el set de entrenamiento considerablemente.
  #Podemos empezar reservando 2000 registros para el test de prueba y ver que onda. Pasariamos de tener 16 mil a 14 mil 
  #registros para el set de entrenamiento, no es una perdida importantisima creo en principio, asi que arrancamos con eso.

  #Por otro lado, nuestro test de prueba deberia tener un 50 50 de Closed Won y Closed Lost, por lo que no podemos elegir asi nomas
  #al azar.

  return df

In [18]:
# Metodo que pasa de DataFrame de Pandas a un DataSet de TensorFlow
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Stage') #Retorna la columna Stage, eliminandolo simultaneamente del DataFrame. 'Stage' seria nuestra columna 'target', es decir, lo que queremos predecir.
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)) #Crea un DataSet cuyos elementos son slices de los tensores pasados a la funcion. Ver documentacion oficial para comprender bien.
  #En pocas palabras, le pasamos las columnas con los datos como diccionario (estilo 'columna':[dato1,dato2,dato3]) y una lista con los resultados estilo [resu1,resu2,resu3].
  #Se genera un DataSet del estilo [('columna':[dato1,dato2,dato3], [resu1, resu2, resu3])].
  #En realidad 'columna' es una lista de todas las columnas con sus correspondientes datos, pero se entiende la idea creo.
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe)) #Al tener el buffer_size del mismo tamanio que la cantidad de datos del dataset, tenemos perfect shuffling (ver documentacion para comprender).
    #Basicamente mezlcamos el dataset para que luego los batches que se armen contengan distintos elementos si lo entrenamos distintas veces.
  ds = ds.batch(batch_size) #Arma batches de tamanio batch_size entre elementos consecutivos del DataSet.
  return ds

In [19]:
#Arma las features, asignando las columnas del DataFrame segun corresponda al tipo de feature
#(numerico, categorico, etc). Entiendase por feature a las columnas del DataFrame.
def set_up_feature_columns(dataframe, numeric_columns, indicator_columns, bucket_columns, crossed_columns, hashed_columns):
  features = []

  #numeric features
  for column_name in numeric_columns:
    features.append(feature_column.numeric_column(column_name))

  #bucket features
  boundaries = [] #En principio este boundary es solo para la columna 'Delta Time'. Ver de como generalizar.
  for i in range(37):
    boundaries.append(i*10.0)

  for column_name in bucket_columns:
    range_column = feature_column.numeric_column(column_name)
    bukect_column = feature_column.bucketized_column(range_column, boundaries)
    features.append(bukect_column)

  #indicator features (one-hot value vector, para aquellas columnas categoricas de pocas opciones)
  for column_name in indicator_columns:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
                                          column_name, dataframe[column_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    features.append(indicator_column)

  #crossed features
  for crossed_feature in crossed_columns:
    categorical_columns = []
    possible_values = 1
    for column_name in crossed_feature:
      column_keys = dataframe[column_name].unique()
      possible_values = possible_values * len(column_keys)
      categorical_columns.append(feature_column.categorical_column_with_vocabulary_list(
                                            column_name, column_keys))
    crossed_feature = feature_column.crossed_column(categorical_columns, hash_bucket_size=possible_values*10) #Ponemos mas valores de los necesarios para tratar de garantizar que no haya colisiones
    features.append(feature_column.indicator_column(crossed_feature))


  #hashed features
  for column_name in hashed_columns:
    hashed_column = feature_column.categorical_column_with_hash_bucket(
                  column_name, hash_bucket_size=100)
    features.append(feature_column.indicator_column(hashed_column))

  return features

Preparamos los features para el modelo, es decir, seteamos cada una de las columnas que vayamos a utilizar del DataFrame. Luego generamos el DataSet en base al DataFrame para darselo como input al modelo.

In [20]:
#Columnas que consideramos numericas
numeric_columns = ['Planned_Opportunity_Time','Total_Taxable_Amount',
                   'Product_Amount','ASP_converted','TRF']
                #,'Expiry_Date_vs_Modified_Date_Difference']
#                   #,'End_Date_vs_Modified_Date_Difference']

#Columnas que consideramos clasificatorias con rango numerico
bucket_columns = []

#Columnas que queremos hashear
hashed_columns = []#'Products']

#Columnas que consideramos categoricas de pocos valores posibles
indicator_columns = [#'Region',
                     'Source',
                     'Account_Type',
                     'Opportunity_Type',
                     #'Delivery_Quarter',
                     #'Has_Expiry_Date',
                     'Brand',
                    #'Delivery_Terms'
                     #'Same_Owner','Same_Owner_Modifier',
                     ]
                      #'Last_Modified_By'
                      #'Product_Family', 'Product_Name',
                      #'Account_Owner', 'Opportunity_Owner', 'Account_Name'
                      #'Territory', 'Billing_Country'

#Columnas crossed, aquellas que queremos un parametro por combinacion
crossed_columns = [[
                    'Pricing_Delivery_Terms_Quote_Appr',
                    'Pricing_Delivery_Terms_Approved'],
                     ['Bureaucratic_Code',
                    'Bureaucratic_Code_0_Approval',
                    'Bureaucratic_Code_0_Approved']]
                   #[],
                   #['Bureaucratic_Code','Bureaucratic_Code_Approved']]

df = pd.read_csv('Train_TP2_Datos_2020-2C.csv')
df = preprocess_dataframe(df)

features = set_up_feature_columns(df,numeric_columns,indicator_columns,bucket_columns, crossed_columns, hashed_columns)

#Separamos el DataFrame en uno de prueba, validacion y entrenamiento
test_lines = 400

np.random.seed(2)
drop_indices = np.random.choice(df.index, test_lines, replace=False)
df_validation = df.loc[drop_indices, :]
df.drop(drop_indices, inplace=True)
df.reset_index(inplace=True, drop=True)

np.random.seed(0)
drop_indices = np.random.choice(df.index, test_lines, replace=False)
df_test = df.loc[drop_indices, :]
df.drop(drop_indices, inplace=True)
df.reset_index(inplace=True, drop=True)

feature_layer = tf.keras.layers.DenseFeatures(features)
ds_training = df_to_dataset(df,batch_size=56, shuffle=False)
ds_validation = df_to_dataset(df_validation, batch_size=56, shuffle=False)
ds_test = df_to_dataset(df_test, batch_size=56, shuffle=False)

df

/home/crispyburritos/anaconda3/envs/default/lib/python3.8/site-packages/pandas/core/series.py:4479: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/home/crispyburritos/anaconda3/envs/default/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,Region,Pricing_Delivery_Terms_Quote_Appr,Pricing_Delivery_Terms_Approved,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Source,Account_Type,Opportunity_Type,Delivery_Terms,...,Total_Taxable_Amount,Stage,Planned_Opportunity_Time,End_Date_vs_Modified_Date_Difference,Expiry_Date_vs_Modified_Date_Difference,Product_Amount,Same_Owner,Same_Owner_Modifier,Delivery_Approved,Bureaucratic_Code_Approved
0,EMEA,1,1,1,1,Bureaucratic_Code_4,None,Account_Type_2,Opportunity_Type_1,Delivery_Terms_2,...,0.547199,0,0.762596,0.548161,1.312943,-0.427738,No,Yes,1,1
1,EMEA,0,0,0,0,Bureaucratic_Code_4,None,Account_Type_2,Opportunity_Type_1,Delivery_Terms_2,...,-0.172044,1,-0.430272,0.472619,-0.479204,-0.427738,No,Yes,1,1
2,Americas,0,0,0,0,Bureaucratic_Code_4,Source_7,Account_Type_5,Opportunity_Type_1,Delivery_Terms_4,...,-0.168476,1,-0.471406,-1.120060,-3.415024,-0.427738,No,Yes,1,1
3,Americas,1,0,1,0,Bureaucratic_Code_5,Source_11,Account_Type_5,Opportunity_Type_19,Delivery_Terms_1,...,0.724658,0,0.721462,0.466324,-0.361299,-0.427738,Yes,Yes,0,0
4,Americas,1,0,1,0,Bureaucratic_Code_5,Source_11,Account_Type_5,Opportunity_Type_19,Delivery_Terms_1,...,1.447064,0,0.083895,0.271174,2.716006,-0.427738,Yes,Yes,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,EMEA,1,0,0,0,Bureaucratic_Code_5,None,Account_Type_2,Opportunity_Type_19,Delivery_Terms_2,...,0.001352,0,0.474662,0.308945,2.326921,-0.427738,Yes,Yes,0,1
8986,EMEA,1,1,0,0,Bureaucratic_Code_4,Source_9,Account_Type_0,Opportunity_Type_1,Delivery_Terms_2,...,-0.123383,1,-0.060072,0.567047,-0.266976,-0.427738,Yes,Yes,1,1
8987,Americas,1,0,0,0,Bureaucratic_Code_5,None,Account_Type_5,Opportunity_Type_19,Delivery_Terms_4,...,2.417765,0,1.338463,0.951053,3.647451,0.162891,Yes,Yes,0,1
8988,EMEA,1,1,1,1,Bureaucratic_Code_4,Source_7,Account_Type_5,Opportunity_Type_1,Delivery_Terms_2,...,-0.137423,1,-0.389139,0.944757,-0.361299,0.753520,Yes,Yes,1,1


Creamos y compilamos el modelo. En esta seccion se tunean las propiedades del modelo.

In [21]:
model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(9, activation='relu'),
    #tf.keras.layers.Dense(3, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

Entrenamos el modelo

In [23]:
#Ignoren el WARNING, esta en la documentacion tambien. Nadie le da bola en StackOverflow xd.
model.fit(ds_training, validation_data=ds_validation, epochs=50)
model.summary()

Epoch 1/50
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Consider rewriting this model with the Functional API.
151/161 [===========================>..] - ETA: 0s - loss: 0.6534 - accuracy: 0.6529

Evaluamos el modelo

In [24]:
print('--- Train ---')
model.evaluate(ds_training)

print('\n--- Validation ---')
model.evaluate(ds_validation)

print('\n--- Test ---')
model.evaluate(ds_test)

--- Train ---
161/161 [==============================] - 0s 3ms/step - loss: 0.4625 - accuracy: 0.7765

--- Validation ---
8/8 [==============================] - 0s 2ms/step - loss: 0.4422 - accuracy: 0.8000

--- Test ---
8/8 [=============================

[0.4880077838897705, 0.7925000190734863]

Escribimos las predicciones

In [25]:
frio_test_df = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
frio_test_df['Stage'] = 'Closed Won' #Esto esta solo para que funque todo, no lo uso. No se bien como armarlo sin los labels de Stage. TODO: Averiguar como es!

aux_df = frio_test_df[['Opportunity_ID']] #Esta columna la vuela el preprocesado sino
frio_test_df = preprocess_dataframe(frio_test_df)
frio_test_ds = df_to_dataset(frio_test_df, shuffle=False, batch_size=56)
predictions = model.predict(frio_test_ds)

aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True) #Lo hacia el preprocesado pero es verdad que lo copie antes a este xd, perdon Agus, paja de dejarlo lindo.
aux_df['Target'] = predictions

aux_df.to_csv('prediccionesFrioFrio.csv', index=False)

'''
df = pd.read_csv('Train_TP2_Datos_2020-2C.csv')
df = df[(df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost')]
df = df[df['Opportunity_ID'] != 9773]
df = df[['Opportunity_ID']]
df.drop_duplicates(subset='Opportunity_ID', inplace=True)
np.random.seed(2)
drop_indices = np.random.choice(df.index, test_lines, replace=False)
df.drop(drop_indices, inplace=True)
df.reset_index(inplace=True, drop=True)
np.random.seed(0)
drop_indices = np.random.choice(df.index, test_lines, replace=False)
df.drop(drop_indices, inplace=True)
predictions = model.predict(ds_training)
df['Target'] = predictions
df.to_csv('prediccionesFrioFrio.csv', index=False)
'''

Consider rewriting this model with the Functional API.


"\ndf = pd.read_csv('Train_TP2_Datos_2020-2C.csv')\ndf = df[(df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost')]\ndf = df[df['Opportunity_ID'] != 9773]\ndf = df[['Opportunity_ID']]\ndf.drop_duplicates(subset='Opportunity_ID', inplace=True)\nnp.random.seed(2)\ndrop_indices = np.random.choice(df.index, test_lines, replace=False)\ndf.drop(drop_indices, inplace=True)\ndf.reset_index(inplace=True, drop=True)\nnp.random.seed(0)\ndrop_indices = np.random.choice(df.index, test_lines, replace=False)\ndf.drop(drop_indices, inplace=True)\npredictions = model.predict(ds_training)\ndf['Target'] = predictions\ndf.to_csv('prediccionesFrioFrio.csv', index=False)\n"